# SIC_Capstone_School

## Agent

In [28]:
class Agent:
    def __init__(self, name):
        self.name = name
    
    def __str__(self):
        return f"{self.name}"

    def describe(self):
        return self.__str__()

## Agent Manager

In [29]:
class AgentManager:
    """Clase base para gestionar la creación y eliminación de agentes."""
    def __init__(self):
        self.agents = {}                    # Diccionario para almacenar agentes
        self.TIME_THRESHOLD = 10            # Umbral de 10 segundos

    def filter_agents(self,*agents_types):
        """Filtra agentes según los tipos proporcionados."""
        filtered_agents = {
            name: agent for name, agent in agents.items()
            if isinstance(agent, tuple(agents_types))      # Filtrar según tipos
        }
        return filtered_agents
    
    def get_agent_by_name(self, agent_name, agent_type):
        """Devuelve el agente con el nombre dado y tipo específico, o None si no se encuentra."""
        return next((agent for agent in agents.values() 
                     if isinstance(agent, agent_type) and agent.name == agent_name), None)

    def add_agent(self, agent_type, agent_name):
        """Añade un nuevo agente al sistema."""
        if isinstance(agent_type, type) and agent_type == Client: 
            agents[agent_name] = Client(agent_name)
        elif isinstance(agent_type, type) and agent_type == School:
            agents[agent_name] = School(agent_name)
        else:
            print(f"Invalid agent type: {agent_type}. Please use valid agent.")
            return
            
        print(f'{agent_type.__name__} {agent_name} added to the system.')

    def remove_agent(self, agent_name):
        """Elimina un agente del sistema."""
        if agent_name in agents:
            del agents[agent_name]
            print(f'Agent {agent_name} removed from the system.')
        else:
            print(f'Agent {agent_name} not found.')

    def list_agents(self, agent_type=None):
        """Muestra todos los agentes o filtra por clientes o ayuntamientos en el sistema."""
        if agent_type:
            print(f"Current {agent_type.__name__}(s):")
            filtered_agents = self.filter_agents(agent_type)
            for agent in filtered_agents.values():               # Imprimir descripciones de los agentes filtrados
                print(agent.describe())
        else:
            print("Current agents:")
            for agent in agents.values():
                print(agent.describe())     

    def check_ready_services(self):
        """Verifica si hay servicios listos para ser atendidos por cada ayuntamiento."""
        filtered_agents = self.filter_agents(School) 
        for school in filtered_agents.values():
            while not school.request_services.is_empty():  # Procesar todos los servicios en la cola
                if self.is_time_to_serve(school):  # Verificar si es tiempo de atender el servicio
                    school.process_request_service()  # Procesar el servicio
                else:
                    break  # Salir si no es tiempo de procesar el siguiente servicio

    def is_time_to_serve(self, school):
        """Verifica si es el momento de atender el siguiente servicio en la cola."""
        if not school.request_services.is_empty():
            last_request = school.request_services.peek()  # Obtenemos la última solicitud
            current_time = time.time()                        # Método que deberías implementar para obtener el tiempo actual
            return (current_time - last_request['timestamp']) >= self.TIME_THRESHOLD  # Define el umbral

        return False        
    
    def validate_client_location(self, client, school_name):
        if client.current_school() is None:
            print(f'Client {client.name} cannot perform this action because not in it.')
            return False
        elif client.current_school() != school_name:
            print(f'Client {client.name} is in a different school: {client.current_school()}.')
            return False
        return True

    def load_agents_from_file(self, file_path):
        """Carga agentes desde un fichero JSON."""

        import json
        try:
            with open(file_path, 'r') as file:
                data = json.load(file)

            # Cargar schools
            for school_name, school_data in data.get("schools", {}).items():
                self.add_agent(School, school_name)
                school = self.get_agent_by_name(school_name, School)
                if school:
                    # Añadir servicios
                    for service in school_data.get("services", []):
                        school.add_service(service)
                    # Añadir solicitudes en cola
                    for request in school_data.get("queue", []):
                        school.add_request_service(request["client_name"], request["service_name"])

            # Cargar clients
            for client_name, client_data in data.get("clients", {}).items():
                self.add_agent(Client, client_name)
                client = self.get_agent_by_name(client_name, Client)
                if client and client_data.get("current_school"):
                    client.enter_school(client_data["current_school"])

            print(f"--- Agents loaded successfully from {file_path}. --- ")
        except FileNotFoundError:
            print(f"Error: File '{file_path}' not found.")
        except json.JSONDecodeError:
            print(f"Error: File '{file_path}' is not a valid JSON file.")
        except Exception as e:
            print(f"An error occurred while loading agents: {e}")     

## Clients

In [ ]:
class Client(Agent):
    """Clase que representa a un cliente que interactúa con el ayuntamiento."""
    def __init__(self, name):
        super().__init__(name)
        self.school_stack = Stack()  # En que ayuntamiento se encuentra
    
    HELP_MESSAGES = {
        "client add_client <client_name>": "Add a client (student) to the system.",
        "client enroll_in_school <client_name> <school_name>": "Enroll a client in an specific school.",
        "client leave_school <client_name>": "Allow a client to leave school.",
        "client join_enrollment_queue <client_name> <school_name><course_name>": "Join a client in a queue to enroll a course.",
        "client assist_course <school_name> <couser_name>": "asistir a un curso en un colegio",
        "client assist_course <school_name> <couser_name>": "asistir a un curso en un colegio",
        "client assist_course <school_name> <couser_name>": "asistir a un curso en un colegio",
        "client take_exam <client_name> <course_name> <exam_name>": "Permitir que un cliente presente un examen de un curso en el que está inscrito.",
        "quit": "q: Exit the simulation."
    }

    @classmethod
    def help(cls):
        """Muestra los comandos disponibles para los clientes."""
        print("Available commands for client:")
        for command, description in cls.HELP_MESSAGES.items():
            print(f"-{command}: {description}")

## School

In [31]:
class School(Agent):
    def __init__(self, name):
        super().__init__(name)
        self.services = []                             
        self.request_services = Queue()  


    HELP_MESSAGES = {
        "school add_school <school_name>": "Add a new school to the system.",
        "school create_course <course_name>": "Create a new course at school.",
        "school show_students <school_name>": "Show the list of all students registered at school.",
        "school show_enrollment_queue <school_name>": "Show the enrollment queue for the course.",
        "school admit_student_from_queue <school_name><course_name>": "Admit the next student from the queue to enroll in to a course.",
        "school show_courses <school_name>": "Mostrar la lista de cursos disponibles en el colegio.",
        "school remove_student <school_name> <client_name>": "Retirar a un cliente del colegio.",
        "school show_list": "muestra la lista de colegios en el sistema",
        "school close / open <schol_name>": "abrir / cerrar colegio, si no hay alumnos en clase",
        "school add_exam_to_course <school_name> <course_name> <exam_name>": "Agregar un examen a un curso específico en el colegio.",
        "school grade_exam <school_name> <course_name> <client_name> <exam_name>": "Calificar un examen que un cliente ha presentado en un curso.",
        "school remove_exam_from_course <school_name> <course_name> <exam_name>": "Eliminar un examen de un curso si ningún cliente lo ha presentado.",
        "school show_exams <school_name> <course_name>": "Mostrar la lista de exámenes disponibles para un curso en el colegio.",
        "quit": "q: Exit the simulation."
    }
    
    def add_school(self, school_name):
        pass
    
    def create_course(self, course_name):
        pass
    
    def show_students(self, school_name):
        pass
    
    def show_enrollment_queue(self, school_name):
        pass
    
    def admit_student_from_queue(self, school_name, course_name):
        pass
    
    def show_courses(self, school_name):
        pass
    
    def remove_student(self, school_name, client_name):
        pass
    
    def show_list(self):
        pass
    
    def close(self, school_name):
        pass
    
    def open(self, school_name):
        pass
    
    def add_exam_to_course(self, school_name, course_name, exam_name):
        pass
    
    def grade_exam(self, school_name, course_name, client_name, exam_name):
        pass
    
    def remove_exam_from_course(self, school_name, course_name, exam_name):
        pass
    
    def show_exams(self, school_name, course_name):
        pass

    @classmethod
    def help(cls):
        """Muestra los comandos disponibles para los ayuntamientos."""
        print("Available commands for school:")
        for command, description in cls.HELP_MESSAGES.items():
            print(f"-{command}: {description}")

## City Simulation

In [32]:
class CitySimulation:
    def __init__(self):
        

        self.agent_manager = AgentManager()              
        self.agent_manager.agents = agents               
         
        self.ERROR_MESSAGES = {                           
            "invalid_command": "Error: Invalid command.",
            "school_not_found": "Error: School '{name}' not found.",
            "client_not_found": "Error: Client '{name}' not found.",
            "service_not_found": "Error: Service '{service}' not found in school '{school}'.",
            "invalid_format": "Error: Invalid command format. Use '{expected_format}'."
        }
        
    def help_school(self):
        """Displays the list of available commands."""
        School.help()

    def help_client(self):
        """Muestra los comandos disponibles para los clientes."""
        Client.help()
        
    def help(self):
        """Displays general help information."""
        print("""
            Available help commands:
            - ? school: Show available commands for schools.
            - ? client: Show available commands for clients.
            - load_agents <file_path>: Load agents from a JSON file.
            - save_agents <file_path>: Save agents to a JSON file.
            - q: Exit the simulation.              
            """)

    def validate_command(self, parts, expected_length, error_key, expected_format):
        if len(parts) != expected_length:
            print(self.ERROR_MESSAGES[error_key].format(expected_format=expected_format))
            return False
        return True
    
    def validate_service_in_school(self, school, service_name):
        if service_name not in school.services:
            print(self.ERROR_MESSAGES["service_not_found"].format(service=service_name, school=school.name))
            return False
        return True
    
    def get_agent_or_error(self, agent_name, agent_type, error_key):
        agent = self.agent_manager.get_agent_by_name(agent_name, agent_type)
        if not agent:
            print(self.ERROR_MESSAGES[error_key].format(name=agent_name))
        return agent

    def command_loop(self):
        """Bucle principal para gestionar comandos del usuario."""
        print("Starting city simulation... Type 'q' to exit")
        while True:
            command = input('> ')
            if command == 'q':
                break
            self.process_command(command)
            self.agent_manager.check_ready_services()        #verifica servicios de los ayuntamientos encolados 

    def process_command(self, command):
        """Procesa los comandos ingresados por el usuario."""
        parts = command.split()
        if not parts:
            return
        cmd = parts[0]
        if cmd == '?':
            if len(parts)==2:
                if parts[1]== 'school':
                    self.help_school()
                elif parts[1]== 'client':
                    self.help_client()
                else:
                    self.help()
            else:
                self.help()            # Llama al método de ayuda
            return
        elif cmd == 'load_agents':
            if self.validate_command(parts, 2, "invalid_format", "load_agents <file_path>"):
                _, file_path = parts
                self.agent_manager.load_agents_from_file(file_path)
        elif cmd == 'school':
            if   parts[1] == 'add_school':
                if self.validate_command(parts, 3, "invalid_format", "school add_school <school_name>"):
                    _, _, school_name = parts
                    self.agent_manager.add_agent(School, school_name)                       
            else:
                print(self.ERROR_MESSAGES["invalid_command"])
                self.help_school()
        elif cmd == 'client':
            if   parts[1] == 'add_client':                  #client add_client <client_name>
                if self.validate_command(parts, 3, "invalid_format", "client add_client <client_name>"):
                    _, _, client_name = parts
                    self.agent_manager.add_agent(Client, client_name)

            else:
                print(self.ERROR_MESSAGES["invalid_format"])
                self.help_client()
        else:
            print("Unknown command. Type 'help' for a list of commands.")

## Stack

In [33]:
class Stack:
    def __init__(self):
        self.stack = []

    def is_empty(self):
        return True if len(self.stack) == 0 else False
    
    def push(self, item):
        self.stack.append(item)
        
    def pop(self):
        return None if self.is_empty() else self.stack.pop()
    
    def peek(self):
        return None if self.is_empty () else self.stack[-1]
    


## Queue

In [34]:
class Queue:
    
    def __init__(self):
        self.queue = []
    
    def is_empty (self):
        return True if len(self.queue) == 0 else False
    
    def peek(self):
        return None if self.is_empty () else self.queue[0]
    
    def enqueue (self, item):
        self.queue.append(item)
    
    def dequeue(self):
        return None if self.is_empty() else self.queue.pop(0)
    
    def size(self):
        return len(self.queue)
    
    def pop(self):
        return None if self.is_empty() else self.stack.pop()
    
    def peek(self):
        return None if self.is_empty () else self.stack[-1]
    


## General agent dictionary

In [35]:
# Diccionario global para almacenar agentes
agents = {}

## Main program

In [36]:
import time
if __name__ == "__main__":
    simulation = CitySimulation()
    simulation.command_loop() 
    

Starting city simulation... Type 'q' to exit
Available commands for school:
-school add_school <school_name>: Add a new school to the system.
-school create_course <course_name>: Create a new course at school.
-school show_students <school_name>: Show the list of all students registered at school.
-school show_enrollment_queue <school_name>: Show the enrollment queue for the course.
-school admit_student_from_queue <school_name><course_name>: Admit the next student from the queue to enroll in to a course.
-school show_courses <school_name>: Mostrar la lista de cursos disponibles en el colegio.
-school remove_student <school_name> <client_name>: Retirar a un cliente del colegio.
-school show_list: muestra la lista de colegios en el sistema
-school close / open <schol_name>: abrir / cerrar colegio, si no hay alumnos en clase
-school add_exam_to_course <school_name> <course_name> <exam_name>: Agregar un examen a un curso específico en el colegio.
-school grade_exam <school_name> <course_n